In [1]:
import numpy as np
import polars as pl
from tqdm import tqdm

from typing import List, Any

import scipy.sparse as sp
from sklearn.model_selection import train_test_split

import random
from collections import Counter

In [2]:
import sys
sys.path.append('graph_recsys')
import network_recsys as nr
from importlib import reload

In [3]:
data = pl.read_parquet('train.parquet')
data.head(2), data.shape

(shape: (2, 2)
 ┌───────┬────────────┐
 │ uid   ┆ friend_uid │
 │ ---   ┆ ---        │
 │ i64   ┆ i64        │
 ╞═══════╪════════════╡
 │ 93464 ┆ 114312     │
 │ 93464 ┆ 103690     │
 └───────┴────────────┘,
 (2872562, 2))

In [4]:
# датафрейм с обратными ребрами
data_rev = (
    data
    .rename({'uid': 'friend_uid', 'friend_uid': 'uid'})
    .select('uid', 'friend_uid')
)

# соединим все в один граф
data = pl.concat([data, data_rev])
del data_rev
data.shape

(5745124, 2)

Данные состоят из двух колонок:

- `uid` – идентификатор пользователя
- `friend_uid` – идентификатор друга этого пользователя

Нашей задачей будет порекомендовать возможных друзей, для оценки вашего решения будет использоваться метрика Recall@10, равная проценту верно угаданных друзей

## Валидация

Так как у нас нет временной последовательности и рекомендации друзей не так сильно зависят от временной составляющей, в качестве можно использовать случайно выбранные ребра в графе (при этом для каждого пользователя будет равная пропорция друзей в валидации, которую можно достичь с помощью stratify параметра)

In [ ]:
reload(nr)
train_df, test_df = nr.prepare_validation_set(data, test_size = 0.1)
train_df.shape, test_df.shape

## Бейзлайн (Random)

In [ ]:
grouped_df = (
    test_df
    .groupby('uid')
    .agg(pl.col('friend_uid').alias('y_rel'))
    .join(
        train_df
        .groupby('uid')
        .agg(pl.col('friend_uid').alias('user_history')),
        'uid',
        how='left'
    )
)

median_seq_len = int(grouped_df['user_history'].apply(len).median())
print(f"среднее число uid в user_history: {median_seq_len}")

In [ ]:
n_users = train_df['uid'].max() + 1

# количество друзей у каждого пользователя
friends_count = np.zeros(n_users)
for uid, count in Counter(train_df['uid']).items():
    friends_count[uid] = count
    
friends_count /= sum(friends_count)

In [ ]:
recall_list = []
recs = np.random.choice(n_users, size=(n_users, TOP_K + median_seq_len), p=friends_count)

for user_id, y_rel, user_history in tqdm(grouped_df.rows()):
    y_rec = [uid for uid in recs[user_id] if uid not in user_history]
    recall_list.append(user_recall(y_rel, y_rec))
    
print(f'Recall@{TOP_K} = {np.mean(recall_list)}')

## DEV
мы хотим, чтобы вершины с похожими
соседями были в векторном
пространстве

In [5]:
reload(nr)
median_count_seq = nr.compute_median_history_count(data)
median_count_seq

35

In [6]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import torch
from torch.nn import CosineSimilarity
from torch.nn import Linear
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid, MovieLens
from torch_geometric.nn import Node2Vec, SAGEConv, LightGCN, to_hetero
from torch_geometric.data import Data, HeteroData
from torch_geometric.utils import degree
import torch_geometric.transforms as T

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
device='cuda'

/home/i.nurgaliev/recsys/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# матрица смежности (пара номеров вершин u и v, между которыми проведены ребра)
edge_index = torch.tensor(data.select(['uid', 'friend_uid']).to_numpy(), dtype=torch.long)

data_network = Data(edge_index=edge_index.T.contiguous())
# сделаем наши ребра ненаправленные
data_network = T.ToUndirected()(data_network)
data_network.validate(raise_on_error=True)

True

In [8]:
model = Node2Vec(
    edge_index=data_network.edge_index,
    embedding_dim=128,  # размер эмбеддинга вершины
    walk_length=30,  # длина случайного блуждания
    context_size=15,  # размер окна из случайного блуждания (как в w2v)
    walks_per_node=20,  # количество случайных блужданий из одной вершины
    num_negative_samples=2,  # количество негативных примеров на один позитивный
    p=1.0,  # параметр вероятности вернуться в предыдущую вершину
    q=0.9,  # параметр вероятности исследовать граф вглубь
    sparse=True,
).to(device)

# класс Node2Vec предоставляет сразу генератор случайного блуждания
loader = model.loader(batch_size=256, shuffle=True, num_workers=8)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        # pos_rw – последовательность из случайного блуждания
        # neg_rw – случайные негативные примеры
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Test training
for epoch in range(1, 30):
    loss = train()
    # acc = test()
    if epoch % 2 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 002, Loss: 1.1315
Epoch: 004, Loss: 1.0824
Epoch: 006, Loss: 1.0756
Epoch: 008, Loss: 1.0752
Epoch: 010, Loss: 1.0752
Epoch: 012, Loss: 1.0750
Epoch: 014, Loss: 1.0748
Epoch: 016, Loss: 1.0746
Epoch: 018, Loss: 1.0740
Epoch: 020, Loss: 1.0739
Epoch: 022, Loss: 1.0737
Epoch: 024, Loss: 1.0734
Epoch: 026, Loss: 1.0733
Epoch: 028, Loss: 1.0733


To predict whether or not two users are friends,
we take the Hadamard product of their node2vec representations and put it through a logistic regression classifier.

In [55]:
reload(nr)
d_train = nr.collect_data_training(data, model, 5, device='cpu')

100%|███████████████████████████████████████████████████████████████████████████████████████| 106563/106563 [00:13<00:00, 8050.66it/s]


In [56]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs', multi_class='ovr')\
                            .fit(d_train['features'],d_train['y_hat'])

In [78]:
all_uid = data['uid'].unique().to_numpy()

all_embs = model.cpu().forward(torch.tensor(all_uid, dtype=torch.long))
TOP_K = 40
n = TOP_K + median_count_seq

In [79]:
device = 'cuda'
model = model.eval().to(device)
cos = CosineSimilarity(dim=1, eps=1e-6)
recs: dict[uid, np.array] = dict()
all_embs = all_embs.to(device)
with torch.no_grad():
    for uid in tqdm(all_uid):
        uid_emb = model.forward(torch.tensor([uid], dtype=torch.long).to(device))
        output = cos(uid_emb, all_embs)
        top_k_indices = torch.topk(output, n)
        recs[uid] = all_uid[top_k_indices.indices.cpu().numpy()]
    

100%|███████████████████████████████████████████████████████████████████████████████████████| 106563/106563 [01:23<00:00, 1283.66it/s]


In [ ]:
reload(nr)
subm = nr.prepare_submission(data, recs)

In [80]:
# Rearrange with another metric
recs2: dict[uid, np.array] = dict()
device = 'cuda'
model = model.eval().to(device)

with torch.no_grad():
    for uid, candidates in tqdm(recs.items()):
        uid_emb = model.forward(torch.tensor([uid], dtype=torch.long).to(device))
        candidates_emb = model.forward(torch.tensor(candidates, dtype=torch.long).to(device))
        res = uid_emb * candidates_emb
        probs = clf.predict_proba(res.cpu())
        prob_friend = probs[:,1]
        ind = np.argsort(-prob_friend)
        recs2[uid] = candidates[ind]

100%|███████████████████████████████████████████████████████████████████████████████████████| 106563/106563 [00:40<00:00, 2658.31it/s]


In [81]:
reload(nr)
assert len(recs2) > 0 and  len(recs) == len(recs2)
subm = nr.prepare_submission(data, recs2, fpath = 'submission2.parquet')

100%|█████████████████████████████████████████████████████████████████████████████████████████| 85483/85483 [00:17<00:00, 4954.50it/s]
